In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Specify the model repository
model_name = "openthaigpt/openthaigpt1.5-7b-instruct"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Use float16 for reduced memory usage
    device_map="auto"        # Automatically place the model on available devices (GPU/CPU)
   
)

# Create a text generation pipeline
qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,          # Adjust based on your needs
    temperature=0.,         # Controls randomness: lower is less random
    top_p=0.95,              # Nucleus sampling
    repetition_penalty=1.2,  # Prevents repetition
)

2024-10-03 01:56:18.880426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 01:56:18.899934: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 01:56:18.905968: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 01:56:18.920917: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-03 01:56:20.325305: W tensorflow/compiler/tf2

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [9]:
def answer_question(question):
    prompt = f"Question: {question}\nAnswer:"
    prompt=question
    print(prompt)
    response = qa_pipeline(prompt )
    print(response)
    answer = response[0]['generated_text'].split("A:")[-1].strip()
    return answer

In [7]:
from IPython.display import display, Markdown
import ipywidgets as widgets

# Create text input widget
question_input = widgets.Text(
    value='',
    placeholder='Type your question here',
    description='Question:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

# Create output area
output_area = widgets.Output()

# Define the function to handle the button click
def on_submit(button):
    with output_area:
        output_area.clear_output()
        user_question = question_input.value
        if user_question.lower() in ["exit", "quit"]:
            display(Markdown("### Session Ended"))
            button.disabled = True
            question_input.disabled = True
        else:
            try:
                answer = answer_question(user_question)
                display(Markdown(f"**Q:** {user_question}\n\n**A:** {answer}\n"))
            except Exception as e:
                display(Markdown(f"An error occurred: {e}"))

# Create submit button
submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='',
    tooltip='Submit your question',
    icon='check'
)

# Link the button click to the function
submit_button.on_click(on_submit)

# Display the widgets
display(question_input, submit_button, output_area)

Text(value='', description='Question:', layout=Layout(width='80%'), placeholder='Type your question here')

Button(description='Submit', icon='check', style=ButtonStyle(), tooltip='Submit your question')

Output()